In [1]:
import cv2

import math
import numpy.linalg as lin

from numpy import *
import numpy as np
from scipy import linalg

def readFile(fileName):
    fin = open(fileName)
    f = fin.read()
    fin.close()
    return f

def readPoints(string):
    
    sourcePoints = [[0 for x in range(20)] for y in range(3)]
    destPoints = [[0 for x in range(20)] for y in range(3)]
    
    i = 0
    for line in string.strip().split("\n"):
        
        points = line.strip().split(",")
        p1 = points[0].strip().split(" ")
        p2 = points[1].strip().split(" ")
        
        sourcePoints[0][i] = float(p1[0])
        sourcePoints[1][i] = float(p1[1])
        sourcePoints[2][i] = 1

        destPoints[0][i] = float(p2[0])
        destPoints[1][i] = float(p2[1])
        destPoints[2][i] = 1
            
        i = i + 1
    return sourcePoints, destPoints


def H_from_points(fp,tp):
    if fp.shape != tp.shape:
        raise RuntimeError('number of points do not match')
        
    # condition points (important for numerical reasons)
    # --from points--
    m = mean(fp[:2], axis=1)
    maxstd = max(std(fp[:2], axis=1)) + 1e-9    #1e-9 niye var?
    C1 = diag([1/maxstd, 1/maxstd, 1]) 
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp = dot(C1,fp)
    
    # --to points--
    m = mean(tp[:2], axis=1)
    maxstd = max(std(tp[:2], axis=1)) + 1e-9
    C2 = diag([1/maxstd, 1/maxstd, 1])
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp = dot(C2,tp)
    
    # create matrix for linear method, 2 rows for each correspondence pair
    nbr_correspondences = fp.shape[1]
    A = zeros((2*nbr_correspondences,9))
    for i in range(nbr_correspondences):        
        A[2*i] = [0,0,0,-fp[0][i],-fp[1][i],-1,
                    tp[1][i]*fp[0][i],tp[1][i]*fp[1][i],tp[1][i]]
            
        A[2*i+1] = [fp[0][i],fp[1][i],1,0,0,0,
                    -tp[0][i]*fp[0][i],-tp[0][i]*fp[1][i],-tp[0][i]]
    
    U,S,V = linalg.svd(A)
    H = V[8].reshape((3,3))    
    
    # decondition
    H = dot(linalg.inv(C2),dot(H,C1))
    
    # normalize and return
    return H / H[2,2]

        
img = cv2.imread("in.jpg", 0)


fContent = readFile("corrs.txt")
srcPoints, destPoints = readPoints(fContent)

srcPoints = np.array(srcPoints)
destPoints = np.array(destPoints)

F = H_from_points(srcPoints,destPoints)


warped = cv2.warpPerspective(img, F, (800, 800))
cv2.imwrite("warped.jpg", warped)

True

True